So... you're providing a library and has dependencies on;
- another library that relies on `PIL.PILLOW_VERSION` and 
- pilow v7.

### Is there a good way to hide the fact that pillow v7 removed `PILLOW_VERSION` from users of your library?

Start by demonstrating the problem using fastai2 source as of 2020-02-20 which uses (see: environment.yml);
- torchvision<=0.4.2
- pillow

In [1]:
try:
    from torchvision.transforms.functional import pad as tvpad
    raise Exception('PILLOW_VERSION should not be available yet')
except ImportError as e:
    print(e)

cannot import name 'PILLOW_VERSION' from 'PIL' (/home/peter/anaconda3/envs/fastai-editable/lib/python3.7/site-packages/PIL/__init__.py)


we could wrap the real import function to add PILLOW_VERSION to PIL 

In [2]:
import builtins
_real_import = builtins.__import__
def _pil_fix_import(name, globals=None, locals=None, fromlist=(), level=0):
    res = _real_import(name, globals, locals, fromlist, level)
    if name == 'PIL': res.PILLOW_VERSION = res.__version__
    return res
builtins.__import__ = _pil_fix_import

no errors this time (o:

In [3]:
from PIL import PILLOW_VERSION

this looks pretty good to me but there are problems with the above approach
- https://docs.python.org/3/library/functions.html#__import__ tells us that doing this is strongly discouraged as it is usually simpler to use import hooks (see PEP 302)
    - maybe I'm missing something because I can't find a simple way to wrap importlib finders/loaders to do the same thing
- you have to import builtins

the following might look more simple but ...
- you would need to run this before anything tries to access `PIL.PILLOW_VERSION`
    - which could be in multiple places and
    - might mean you sometimes import PIL when you don't need it and
- you have to import PIL, rather than PIL.Image etc

In [4]:
import PIL
PIL.PILLOW_VERSION = PIL.__version__

Note: if you're writing a notebook, using the code in the cell above could be a good alternative to using a pre-v7 version of pillow